#Toronto Neighborhoods Segmentation and Clustering

#Installing dependencies

In [1]:

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json
from bs4 import BeautifulSoup

from geopy.geocoders import Nominatim #convert adrresses to longitude and latitude

import requests

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

ModuleNotFoundError: No module named 'bs4'

Fetching Toronto Neighborhood data from Wikipedia¶

In [ ]:
#getting html
wiki_html = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

#converting to beautifulsoup
soup = BeautifulSoup(wiki_html.text)

#finding table elements
table = soup.find_all('table')

#using pandas built in function to read tables from html
df = pd.read_html(str(table))[0]

Cleaning the data

In [ ]:
#if borough present AND neighborhood not present, neighborhodo = borough
for index, row in df.iterrows():
    if row["Borough"] != "Not assigned" and row['Neighbourhood'] == "Not assigned":
        row['Neighbourhood'] = row["Borough"]

#removing rows with Neighborhood = "Not assigned"
df = df[df.Neighbourhood != "Not assigned"]

#resetting index
df.reset_index(drop = True, inplace = True

In [ ]:
df.head()

Adding Latitude and Longitudes for each Postal Code by merging with CSV file

In [ ]:
csv = pd.read_csv("Geospatial_Coordinates.csv")
csv.head()

Creating a map of Toronto neighborhoods

In [ ]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

In [ ]:
# create map of Toronto using latitude and longitude values
map = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map)  
    
map

Filtering to only show boroughs of Toronto

In [ ]:
toronto_data = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_data.head()

In [ ]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Borough'], toronto_data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Defining Foursqaure Credentials¶

In [ ]:
#using python-dotenv to protect Foursqaure credentials
%load_ext dotenv
%dotenv
import os

CLIENT_ID = os.getenv("client_ID") # your Foursquare ID
CLIENT_SECRET = os.getenv("client_secret") # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID SIZE: ' + str(len(CLIENT_ID)))
print('CLIENT_SECRET SIZE: ' + str(len(CLIENT_SECRET)))

Exploring the first neighborhood in the dataset

In [ ]:
toronto_data.loc[0, 'Neighbourhood']

In [ ]:
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

In [ ]:
RADIUS = 500

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    RADIUS, 
    LIMIT)

url # display URL

In [ ]:
results = requests.get(url).json()
results

We need to define a function that can extract the category of a venue

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']


Now we can clean and read the JSON response into a pandas dataframe

In [ ]:
#fetching the items list from response JSON
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues = nearby_venues.rename(columns = {"name":"Name", "categories":"Categories", "lat":"Latitude", "lng":"Longitude"})

nearby_venues.head()

How many points of interests were returned by the Foursqaure API?

In [ ]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

Exploring all the neighborhoods in the dataset
Let's create a function to repeat the same process to all the neighborhoods in Manhattan

In [ ]:
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

And run this function to get points of interest for all the neighborhoods in the dataset

In [ ]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [ ]:
print(toronto_venues.shape)
toronto_venues.head()


Let's check how many venues were returned for each neighborhood

In [ ]:
toronto_venues.groupby('Neighborhood').count()

Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

Analyzing each neighborhood
We can do one-hot encoding to convert our variables into integers that we can easily analyze with our algorithms

In [ ]:

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
toronto_onehot = toronto_onehot[ ['Neighborhood'] + [ col for col in toronto_onehot.columns if col != 'Neighborhood' ] ]

toronto_onehot.head()

And let's examine the new dataframe size.

In [ ]:
toronto_onehot.shape

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

Let's confirm the new size

In [ ]:
toronto_grouped.shape

Let's print each neighborhood along with the top 5 most common venues

In [ ]:

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')


Let's put that into a pandas dataframe

First, let's write a function to sort the venues in descending order.


In [ ]:
row_categories = row.iloc[1:]
row_categories_sorted = row_categories.sort_values(ascending=False)
    
return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [ ]:

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted = neighborhoods_venues_sorted.rename(columns = {"Neighborhood":"Neighbourhood"})

neighborhoods_venues_sorted.head()

Performing K-means clustering on the neighborhoods

In [ ]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

Finally, let's visualize the resulting clusters

In [ ]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Make this Notebook Trusted to load map: File -> Trust Notebook

Extracting meaning from the clusters

In [ ]:
# Cluster 0:

toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 0 shows a lot of coffee shops in the most common venue column. This means these locations are popular for their coffee shops and cafes. Seeing the map, we can see this cluster is usually in busy areas in the middle of the city, where we expect such cafes to be popular.
This makes sense, considering Toronto is a hub of young, skilled workers working in offices. We'd assume they'd like cafes, as I do too :D

In [ ]:
# Cluster 1

toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


Cluster 1 has two points, rows 21 and 33 near to a small-ish park. This suggests this cluster represents in-city parks. However row 4 seems almost an outlier, I suspect it is because the spot is near a Trail.

In [ ]:
# Cluster 2

toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 2 only has one point and it doesnt seem special in any way

In [ ]:
# Cluster 3

toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 3 is just dominated by the lawrence park

In [ ]:
# Cluster 4

toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

Cluster 4 seems popular for its dive bar and music venues, so it is more a social area with good nightlife